In [1]:
%load_ext autoreload
%autoreload 1

In [34]:
import gym
import torch
import collections
import numpy as np
import itertools
import math
import copy
import random

env = gym.make("CartPole-v0")#.unwrapped

n_input = env.observation_space.shape[0]
n_output = env.action_space.n

def _make_namer():
    tbl = dict()
    seen = set()
    def namer(name):
        if name in tbl:
            tbl[name] += 1
        else:
            tbl[name] = 0
        ret = f"{name}_{tbl[name]}"
        assert ret not in seen
        seen.add(ret)
        return ret
    return namer


namer = _make_namer()
act = torch.nn.Tanh
var = torch.autograd.Variable
ftn = torch.FloatTensor
ltn = torch.LongTensor
btn = torch.ByteTensor

def _conf_of(**kwargs):
    return collections.namedtuple("_Conf", kwargs.keys())(**kwargs)


args = _conf_of(
    n_middle = 50,
    lr = 1e-2,
    gamma = 0.95,
    n_batch = 32,
    n_episodes=2000,
    n_start_train=500,
    n_target_update_interval=100,
    n_steps=200,
    epsilon=0.3,
)


model = torch.nn.Sequential(collections.OrderedDict((
    (namer("fc"), torch.nn.Linear(n_input, args.n_middle)),
    (namer("ac"), act()),
    (namer("fc"), torch.nn.Linear(args.n_middle, args.n_middle)),
    (namer("ac"), act()),
    ("output", torch.nn.Linear(args.n_middle, n_output)),
)))
def _init(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.kaiming_uniform(m.weight.data)
        m.weight.data.mul_(math.sqrt(1/2))
        m.bias.data.fill_(0)
model.apply(_init)
target_model = copy.deepcopy(model)
#opt = torch.optim.Adam(model.parameters(), lr=args.lr)
opt = torch.optim.Adam(model.parameters(), eps=1e-2)
#opt = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.1)
#opt = torch.optim.SGD(model.parameters(), lr=args.lr)

buffer = []
episode_result_list = []
i_total_step = -1
for i_episode in range(1, args.n_episodes + 1):
    si = env.reset()
    step_result_list = []
    for i_step in range(1, args.n_steps + 1):
        i_total_step += 1
        
        if random.random() < args.epsilon:
            ai1 = env.action_space.sample()
        else:
            model.eval()
            ai1 = int(model(var(ftn([si]), volatile=True)).max(1)[1].data.numpy()[0])
        si1, ri1, done, debug_info = env.step(ai1)
        buffer.append(dict(si=si, ai1=ai1, ri1=ri1, si1=si1, done=done))
        metric = None
        if i_total_step > args.n_start_train:
            batch = random.sample(buffer, args.n_batch)
            batch = dict(
                si=ftn([t["si"] for t in batch]),
                ai1=ltn([t["ai1"] for t in batch]).view(-1, 1),
                ri1=ftn([t["ri1"] for t in batch]).view(-1, 1),
                si1=ftn([t["si1"] for t in batch]),
                mask=ftn([not t["done"] for t in batch]).view(-1, 1),
            )
            model.train()
            q_pred = model(var(batch["si"])).gather(1, var(batch["ai1"]))
            target_model.eval()
            model.eval()
            mask = var(batch["mask"], volatile=True)
            #print("si", batch["si"])
            #print("si1", batch["si1"])
            q_target = (
                var(batch["ri1"], volatile=True)
                + args.gamma*mask*target_model(var(batch["si1"], volatile=True)).gather(1, model(var(batch["si1"], volatile=True)).max(1)[1].view(-1, 1))
            )
            q_target.volatile = False
            loss = torch.nn.SmoothL1Loss()(q_pred, q_target)
            opt.zero_grad()
            loss.backward()
            opt.step()
        if metric is not None:
            q_list.append(np.mean(metric["q_pred"]))
            step_result_list.append(metric)
        if i_total_step%(args.n_target_update_interval) == 0:
            target_model = copy.deepcopy(model)
        if done:
            break
        si = si1
    episode_result_list.append(step_result_list)
    print(i_step, end="\t")
    if i_episode%10 == 0:
        print()

51	58	39	44	43	39	51	38	35	39	
52	56	16	12	10	51	9	17	39	35	
23	37	26	53	60	63	50	62	77	35	
114	95	121	86	100	72	138	52	54	186	
86	51	137	97	30	36	88	57	57	101	
200	200	164	152	131	132	200	200	158	160	
160	78	200	200	160	189	200	176	200	192	
167	143	22	133	48	200	143	181	200	200	
200	192	200	184	200	200	200	177	200	200	
200	200	168	200	200	200	144	200	180	186	
37	200	200	165	200	176	200	200	110	186	
189	200	200	200	200	200	200	200	200	200	
200	200	200	175	200	200	200	200	200	200	
199	197	200	194	200	104	186	200	200	200	
190	200	200	113	200	200	200	173	200	200	
181	200	99	200	197	200	200	53	200	194	
22	200	200	200	200	200	200	200	200	25	
200	64	189	200	200	24	200	200	200	200	
200	200	200	50	200	39	200	200	200	200	
16	200	200	200	200	200	200	200	200	200	
200	200	200	200	200	200	200	200	171	159	
200	200	200	37	67	200	200	200	200	200	
78	200	200	200	200	200	200	200	200	200	
148	200	200	200	141	200	200	200	200	200	
200	200	200	200	200	200	42	200	200	121	
200	200	200	200	200	200	200	68	167	2

KeyboardInterrupt: 

In [5]:
model.apply(_init)

Sequential(
  (fc_0): Linear(in_features=4, out_features=50)
  (ac_0): Tanh()
  (fc_1): Linear(in_features=50, out_features=50)
  (ac_1): Tanh()
  (output): Linear(in_features=50, out_features=2)
)

In [8]:
model.fc_0.weight.data.numpy().var()

0.25241929

In [9]:
model.fc_1.weight.data.numpy().var()

0.020535447

In [11]:
model.output.weight.data.numpy().var()

0.018865753